# Appendix 10.2: Tool Use

- [Lesson](#lesson)
- [Exercises](#exercises)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
# Import python's built-in regular expression library
import re
from ollama import chat, ChatResponse, Options


# Stores the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME

def get_completion(prompt: str, system_prompt="", prefill="", tools=None):
    # Ensure tools is a list, even if None is passed
    tools = tools if tools is not None else []
    
    response = chat(
        model=MODEL_NAME,
        options=Options(
            max_tokens=2000,
            temperature=0.0,
            num_ctx=8192 # larger context defined to include large prompt in this chapter, default value is 2048
        ),
        messages=[
            {"role": "system", "content": system_prompt},  
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": prefill}
        ],
        tools=tools,
    )
    return response

---

## Lesson

While it might seem conceptually complex at first, tool use, a.k.a. function calling, is actually quite simple! You already know all the skills necessary to implement tool use, which is really just a combination of substitution and prompt chaining.

In previous substitution exercises, we substituted text into prompts. With tool use, we substitute tool or function results into prompts. Ollama can't literally call or access tools and functions. Instead, we have Ollama:
1. Output the tool name and arguments it wants to call
2. Halt any further response generation while the tool is called
3. Then we reprompt with the appended tool results

Function calling is useful because it expands Ollama's capabilities and enables Ollama to handle much more complex, multi-step tasks.
Some examples of functions you can give Ollama:
- Calculator
- Word counter
- SQL database querying and data retrieval
- Weather API

You can get Ollama to do tool use in two way:
- Manual
- Automatic
  
Python functions or any other functions you want to call can be passed to the model through the tools parameter of the chat API.

### Examples

To enable tool use in Ollama, we start with defining the functions we are going to use. 

In [ ]:
def add_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers

  Args:
    a: The first integer number
    b: The second integer number

  Returns:
    int: The sum of the two numbers
  """
  return a + b

def subtract_two_numbers(a: int, b: int) -> int:
  """
  Subtract two numbers
  """
  return a - b

Now we add them to a dictionary to be able to call them when Model will ask us to.

In [ ]:
available_functions = {
  'add_two_numbers': add_two_numbers,
  'subtract_two_numbers': subtract_two_numbers,
}

Here the manual creation of JSON schema for the subtract_two_numbers function, while for add_two_numbers we will use automatic way.

In [ ]:
# Tools can still be manually defined and passed into chat
subtract_two_numbers_tool = {
  'type': 'function',
  'function': {
    'name': 'subtract_two_numbers',
    'description': 'Subtract two numbers',
    'parameters': {
      'type': 'object',
      'required': ['a', 'b'],
      'properties': {
        'a': {'type': 'integer', 'description': 'The first number'},
        'b': {'type': 'integer', 'description': 'The second number'},
      },
    },
  },
}

Next, use the *tools* field to pass the functions as tools to Ollama and we use the returned tool call and arguments provided by the model to call the respective function.

In [ ]:
# Prompt
PROMPT = "What is 2015 + 20?"
# Get Ollama's response
response = get_completion(PROMPT, tools=[add_two_numbers,subtract_two_numbers])

for tool in response.message.tool_calls or []:
  function_to_call = available_functions.get(tool.function.name)
  if function_to_call:
    result = function_to_call(**tool.function.arguments)
  else:
    print('Function not found:', tool.function.name)

print(result)

Let's chain the result for the first call with another one:

In [ ]:
# Prompt
PROMPT = f"{result} - 10?"

# Get Ollama's response
response = get_completion(PROMPT, tools=[add_two_numbers,subtract_two_numbers])

for tool in response.message.tool_calls or []:
  function_to_call = available_functions.get(tool.function.name)
  if function_to_call:
    result = function_to_call(**tool.function.arguments)
  else:
    print('Function not found:', tool.function.name)

print(result)

### Pass existing functions as tools

Functions from existing Python libraries, SDKs, and elsewhere can now also be provided as tools. For example, the following code passes the request function from the requests library as a tool to fetch the contents of the Ollama website:

In [ ]:
import requests

available_functions = {
  'request': requests.request,
}

PROMPT = "get the ollama.com webpage?"

# Print Ollama's response
response = get_completion(PROMPT, tools=[requests.request])

In [ ]:
for tool in response.message.tool_calls or []:
  function_to_call = available_functions.get(tool.function.name)
  if function_to_call == requests.request:
    # Make an HTTP request to the URL specified in the tool call
    resp = function_to_call(
      method=tool.function.arguments.get('method'),
      url=tool.function.arguments.get('url'),
    )
    print(resp.text)
  else:
    print('Function not found:', tool.function.name)


---

## Exercises
- [Exercise 10.2.1 - SQL](#exercise-1021---SQL)

### Exercise 10.2.1 - SQL
In this exercise, you'll be writing a tool use prompt for querying and writing to the world's smallest "database". Here's the initialized database, which is really just a dictionary.

In [ ]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

And here is the code for the functions that write to and from the database.

In [ ]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

To solve the exercise, start by defining a system prompt like `system_prompt_tools_specific_tools` above. Make sure to include the name and description of each tool, along with the name and type and description of each parameter for each function. We've given you some starting scaffolding below.

When you're ready, you can try out your tool definition system prompt on the examples below. Just run the below cell!

In [ ]:
examples = [
    "Add a user to the database named Debora with email debora@example.com.",
    "Add a product to the database named Thingo with price 10",
    "Tell me the name of User 2 from the database",
    "Tell me the name of Product 3 from the database"
]

available_functions = {
    'get_user': get_user,
    'get_product': get_product,
    'add_user': add_user,
    'add_product': add_product,
}

for example in examples:
    # Get & print Ollama's response
    function_calling_response = get_completion(example, tools=[get_user, get_product, add_user, add_product])
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

If you did it right, the function calling messages should call the `add_user`, `add_product`, `get_user`, and `get_product` functions correctly.

For extra credit, add some code cells and write parameter-parsing code. Then call the functions with the parameters Ollama gives you to see the state of the "database" after the call.

❓ If you want to see a possible solution, run the cell below!

In [ ]:
from hints import exercise_10_2_1_solution; print(exercise_10_2_1_solution)

### Congrats!

Congratulations on learning tool use and function calling! 
For more information on how to leverage function calling go to blog post **[Ollama Python library 0.4 with function calling improvements](https://ollama.com/blog/functions-as-tools)**